<a href="https://colab.research.google.com/github/SamuelGeron/gcpDW/blob/main/Transform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create Dimensions and Fact Tables

In [111]:
import pandas as pd
from astropy.table import Table
import numpy as np
import re
import locale
from pandas.tseries.offsets import DateOffset
from datetime import datetime
from pandas.io import gbq
from google.colab import auth
auth.authenticate_user()
project_id = 'animated-falcon-292800'
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

# Payments

#Dimensions

## Plano

In [112]:
#Query Plano from Big Query
dim_pagamentos_plano = client.query('''

SELECT 
  DISTINCT
  ROW_NUMBER() OVER (ORDER BY Plano ASC) as PlanoID,
  Plano,
  Valor
FROM `animated-falcon-292800.extract.pagamentos`
GROUP BY Plano, Valor
ORDER BY Plano

''').to_dataframe();

#Store Plano Dimension back to Big Query
dim_pagamentos_plano.to_gbq(destination_table='transform.Dim_Pagamentos_Plano',
                                project_id='animated-falcon-292800',
                                if_exists='replace')

1it [00:04,  4.39s/it]


# Facts

## Query Data from Big Query

In [113]:
fact_pagamentos = client.query('''

SELECT 
  pag.ClienteID,
  pag.DataDoPagamento,
  pag.Valor,
  pag.Plano,
  dp.PlanoID
FROM `animated-falcon-292800.extract.pagamentos` pag
LEFT JOIN `animated-falcon-292800.transform.Dim_Pagamentos_Plano` dp
  ON pag.Plano = dp.Plano

''').to_dataframe();

#Indexing Data
fact_pagamentos['PagamentosID'] = np.arange(len(pagamentos))

In [114]:
#Add months to date
def add_months_to_date(date, months_to_add):
  formatted_date = pd.to_datetime(date, format='%d/%m/%Y')
  formatted_date = formatted_date + DateOffset(months=(months_to_add))
  return formatted_date.strftime("%d/%m/%Y")

## Create a new table for pricing plan values


In [115]:
#Creates another table with generated values based on SaaS Pricing Plan
fact_pagamentos_vigencia = Table(names=('PagamentosID', 'DataVigencia', 'ValorVigencia'),dtype=('int', 'str', 'float'))
for p in range(0, fact_pagamentos.Plano.size):
  n_vigencia = int(re.findall("[^/]*$", fact_pagamentos.Plano[p])[0]) + 1 
  for n in range(1, n_vigencia):
    #fix currency value
    clean = fact_pagamentos.Valor[p][-(len(fact_pagamentos.Valor[p])-3):]
    value = float(clean.replace(".","").replace(",","."))
    #add row to new table
    fact_pagamentos_vigencia.add_row((p,
                     add_months_to_date(fact_pagamentos.DataDoPagamento[p], n-1),
                     value/(n_vigencia-1)))

## Store facts in Big Query

In [116]:
#Vigencia
fact_pagamentos_vigencia.to_pandas().to_gbq(destination_table='transform.Fact_Pagamentos_Vigencia',
                                            project_id='animated-falcon-292800',
                                            if_exists='replace')

1it [00:04,  4.36s/it]


In [117]:
#Pagamentos
fact_pagamentos.drop(columns=['Plano', 'Valor'])
fact_pagamentos.to_gbq(destination_table='transform.Fact_Pagamentos',
                                project_id='animated-falcon-292800',
                                if_exists='replace')

1it [00:04,  4.34s/it]


# Clients

# Dimensions

## Localizacao

In [118]:
#Query Clients location from Big Query
dim_clientes_localizacao = client.query('''

SELECT 
  DISTINCT
  ROW_NUMBER() OVER (ORDER BY cidade ASC) as LocalID,
  cidade,
  estado
FROM `animated-falcon-292800.extract.clientes`
GROUP BY cidade, estado
ORDER BY cidade

''').to_dataframe();

#Store Dimension table in big query
dim_clientes_localizacao.to_gbq(destination_table='transform.Dim_Clientes_Localizacao',
                                project_id='animated-falcon-292800',
                                if_exists='replace')

1it [00:05,  5.54s/it]


## Segmento

In [119]:
#Query Clients segment from Big Query
dim_clientes_segmento = client.query('''

SELECT 
  DISTINCT
  ROW_NUMBER() OVER (ORDER BY segmento ASC) as SegmentoID,
  segmento
FROM `animated-falcon-292800.extract.clientes`
GROUP BY segmento

''').to_dataframe();

#Store Dimension table in big query
dim_clientes_segmento.to_gbq(destination_table='transform.Dim_Clientes_Segmento',
                                project_id='animated-falcon-292800',
                                if_exists='replace')

1it [00:02,  2.44s/it]


## Store table with dimensions IDs


In [120]:
#Query Client from Big Query and apply its respectives IDs for dimensions
dim_clientes = client.query('''

SELECT 
  id AS ClienteID,
  nome as Empresa,
  loc.LocalID,
  seg.SegmentoID
FROM `animated-falcon-292800.extract.clientes` cli
LEFT JOIN `animated-falcon-292800.transform.Dim_Clientes_Localizacao` loc
  ON cli.cidade = loc.cidade
LEFT JOIN `animated-falcon-292800.transform.Dim_Clientes_Segmento` seg
  ON cli.segmento = seg.segmento

''').to_dataframe();

#Store Dimension table in big query
dim_clientes.to_gbq(destination_table='transform.Dim_Clientes',
                                project_id='animated-falcon-292800',
                                if_exists='replace')

1it [00:04,  4.76s/it]
